In [1]:
%pylab
%matplotlib inline
%load_ext watermark
%watermark -v -m -p numpy,tensorflow

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib
CPython 3.5.6
IPython 6.5.0

numpy 1.15.2
tensorflow 1.11.0

compiler   : GCC 7.3.0
system     : Linux
release    : 4.15.0-34-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 4
interpreter: 64bit


텐서플로 1.7.0 버전에서부터는 샘플 데이터를 다운로드하는 기능이 제외될 예정이라는 경고가 발생합니다. 경고를 무시하기 위해 로깅 설정을 변경합니다.

In [2]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [3]:
import time
import tensorflow as tf
from tensorflow.contrib.rnn import BasicLSTMCell, MultiRNNCell, DropoutWrapper

import reader

Small Config 정보를 사용합니다.

In [4]:
class SmallConfig(object):
    """Small config."""
    init_scale = 0.1
    learning_rate = 1.0
    max_grad_norm = 5
    num_layers = 2
    num_steps = 20
    hidden_size = 200
    max_epoch = 4
    max_max_epoch = 13
    keep_prob = 1.0
    lr_decay = 0.5
    batch_size = 20
    vocab_size = 10000

트레이닝과 테스트에 사용할 두개의 config 오브젝트를 만듭니다.

In [5]:
config = SmallConfig()
eval_config = SmallConfig()
eval_config.batch_size = 1
eval_config.num_steps = 1

PTB 모델을 만들어 주는 클래스를 작성합니다.

In [6]:
class PTBModel(object):
    """The PTB model."""

    def __init__(self, config, is_training=False):
        self.batch_size = config.batch_size
        self.num_steps = config.num_steps
        input_size = [config.batch_size, config.num_steps]
        self.input_data = tf.placeholder(tf.int32, input_size)
        self.targets = tf.placeholder(tf.int32, input_size)

        lstm_fn = lambda: BasicLSTMCell(config.hidden_size, forget_bias=0.0, state_is_tuple=True, 
                                        reuse=tf.get_variable_scope().reuse)
        # SmallConfig에서는 드롭아웃이 적용되지 않습니다.
        if is_training and config.keep_prob < 1:
            lstm_fn = lambda: DropoutWrapper(lstm_fn(), config.keep_prob)
        # 두개의 계층을 가진 신경망 구조를 만듭니다.
        cell = MultiRNNCell([lstm_fn() for _ in range(config.num_layers)], state_is_tuple=True)

        self.initial_state = cell.zero_state(config.batch_size, tf.float32)

        with tf.device("/cpu:0"):
            embedding_size = [config.vocab_size, config.hidden_size]
            embedding = tf.get_variable("embedding", embedding_size)
            inputs = tf.nn.embedding_lookup(embedding, self.input_data)

        # SmallConfig에서는 드롭아웃이 적용되지 않습니다.
        if is_training and config.keep_prob < 1:
            inputs = tf.nn.dropout(inputs, config.keep_prob)

        # 각 배치마다 순서대로 데이터를 뽑아 셀에 입력합니다. 
        outputs = []
        state = self.initial_state
        with tf.variable_scope("RNN"):
            for time_step in range(config.num_steps):
                if time_step > 0: tf.get_variable_scope().reuse_variables()
                (cell_output, state) = cell(inputs[:, time_step, :], state)
                outputs.append(cell_output)

        # output의 크기를 20x20x200에서 400x200으로 변경합니다.
        output = tf.reshape(tf.concat(outputs, 1), [-1, config.hidden_size])
        softmax_w_size = [config.hidden_size, config.vocab_size]
        softmax_w = tf.get_variable("softmax_w", softmax_w_size)
        softmax_b = tf.get_variable("softmax_b", [config.vocab_size])
        # logits의 크기는 400x10000이 됩니다.
        logits = tf.matmul(output, softmax_w) + softmax_b
    
        loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example(
            [logits],
            [tf.reshape(self.targets, [-1])],
            [tf.ones([config.batch_size * config.num_steps])])
        self.cost = tf.reduce_sum(loss) / config.batch_size
        self.final_state = state

        if not is_training:
            return

        self.lr = tf.Variable(0.0, trainable=False)
        tvars = tf.trainable_variables()
        # 기울기 클리핑을 수행합니다.
        grads, _ = tf.clip_by_global_norm(tf.gradients(self.cost, tvars),
                                          config.max_grad_norm)
        optimizer = tf.train.GradientDescentOptimizer(self.lr)
        self.train_op = optimizer.apply_gradients(zip(grads, tvars))

    def assign_lr(self, session, lr_value):
        session.run(tf.assign(self.lr, lr_value))

에포크를 처리할 함수를 만듭니다.

In [7]:
def run_epoch(session, m, data, is_training=False):
    """Runs the model on the given data."""
    epoch_size = ((len(data) // m.batch_size) - 1) // m.num_steps
    start_time = time.time()
    costs = 0.0
    iters = 0
    
    eval_op = m.train_op if is_training else tf.no_op()
    
    # initial_state는 2x20x200 크기의 튜플입니다.
    state_list = []
    for c, h in m.initial_state:
        state_list.extend([c.eval(), h.eval()])
    
    ptb_iter = reader.ptb_iterator(data, m.batch_size, m.num_steps)
    for step, (x, y) in enumerate(ptb_iter):
        fetch_list = [m.cost]
        # final_state 튜플에 담겨있는 상태를 꺼내어 fetch_list에 담습니다. 
        for c, h in m.final_state:
            fetch_list.extend([c, h])
        fetch_list.append(eval_op)
        
        # 이전 스텝에서 구해진 state_list가 feed_dict로 주입됩니다.
        feed_dict = {m.input_data: x, m.targets: y}
        for i in range(len(m.initial_state)):
            c, h = m.initial_state[i]
            feed_dict[c], feed_dict[h] = state_list[i*2:(i+1)*2]
        
        # fetch_list에 담긴 final_state의 결과를 state_list로 전달 받습니다.
        cost, *state_list, _ = session.run(fetch_list, feed_dict)

        costs += cost
        iters += m.num_steps

        if is_training and step % (epoch_size // 10) == 10:
            print("%.3f perplexity: %.3f speed: %.0f wps" %
                    (step * 1.0 / epoch_size, np.exp(costs / iters),
                     iters * m.batch_size / (time.time() - start_time)))

    return np.exp(costs / iters)

In [8]:
raw_data = reader.ptb_raw_data('simple-examples/data')
train_data, valid_data, test_data, _ = raw_data

train_data, valid_data, test_data 는 단어를 숫자로 바꾼 리스트입니다.  
가장 많이 나온 단어 순으로 0번 부터 시작하여 10000번 까지의 번호를 가지고 있습니다.

In [9]:
with tf.Graph().as_default(), tf.Session() as session:
    initializer = tf.random_uniform_initializer(-config.init_scale, config.init_scale)

    # 학습과 검증, 테스트를 위한 모델을 만듭니다.
    with tf.variable_scope("model", reuse=None, initializer=initializer):
        m = PTBModel(config, is_training=True)
    with tf.variable_scope("model", reuse=True, initializer=initializer):
        mvalid = PTBModel(config)
        mtest = PTBModel(eval_config)
        
    tf.global_variables_initializer().run()
    
    for i in range(config.max_max_epoch):
        # lr_decay는 반복속도를 조절해 주는 역할을 합니다.
        lr_decay = config.lr_decay ** max(i - config.max_epoch, 0.0)
        m.assign_lr(session, config.learning_rate * lr_decay)
        print("Epoch: %d Learning rate: %.3f" % (i + 1, session.run(m.lr)))
        
        perplexity = run_epoch(session, m, train_data, is_training=True)
        print("Epoch: %d Train Perplexity: %.3f" % (i + 1, perplexity))

        perplexity = run_epoch(session, mvalid, valid_data)
        print("Epoch: %d Valid Perplexity: %.3f" % (i + 1, perplexity))

    perplexity = run_epoch(session, mtest, test_data)
    print("Test Perplexity: %.3f" % perplexity)

Epoch: 1 Learning rate: 1.000
0.004 perplexity: 5031.668 speed: 10529 wps
0.104 perplexity: 832.873 speed: 31272 wps
0.204 perplexity: 617.159 speed: 32771 wps
0.304 perplexity: 498.391 speed: 33415 wps
0.404 perplexity: 430.419 speed: 33594 wps
0.504 perplexity: 386.209 speed: 33575 wps
0.604 perplexity: 347.987 speed: 33671 wps
0.703 perplexity: 321.971 speed: 33664 wps
0.803 perplexity: 301.308 speed: 33779 wps
0.903 perplexity: 282.299 speed: 33901 wps
Epoch: 1 Train Perplexity: 268.141
Epoch: 1 Valid Perplexity: 178.698
Epoch: 2 Learning rate: 1.000
0.004 perplexity: 210.388 speed: 26919 wps
0.104 perplexity: 151.180 speed: 34528 wps
0.204 perplexity: 158.486 speed: 34726 wps
0.304 perplexity: 153.300 speed: 34597 wps
0.404 perplexity: 150.206 speed: 34492 wps
0.504 perplexity: 147.746 speed: 34299 wps
0.604 perplexity: 143.027 speed: 34272 wps
0.703 perplexity: 140.945 speed: 34233 wps
0.803 perplexity: 138.980 speed: 34257 wps
0.903 perplexity: 135.458 speed: 34257 wps
Epoch: 2 